# Creating a robot object to access services with

https://dev.bostondynamics.com/docs/python/understanding_spot_programming#create-the-sdk-object

Start by creating an sdk object and robot for a given ip.

In [1]:
import bosdyn.client
sdk = bosdyn.client.create_standard_sdk('understanding-spot')

In [2]:
robot = sdk.create_robot('192.168.80.3')

The robot-id service provides metadata about the robot. This includes:
-serial number
- Type of robot (eg: spot)
- Robot version
- Version information
- Nickname
- Computer serial number

The robot.ensure_client() method will create an object for calling the service.
Calling id_client.get_id() will execute the RPC and is blocking. It accepts an optional timeout argument.

In [5]:
id_client = robot.ensure_client('robot-id')
id_client.get_id()

UnableToConnectToRobotError: _InactiveRpcError: The robot may be offline or otherwise unreachable.

All RPC calls require authentication, except for get_id().
The python API wraps around the RPC calls, meaning you don't have to set all the protobuf fields yourself.
Calling robot.authentica(...) will call the required authenticate service and get the credentials to provide with further RPC calls.

In [ ]:
robot.authenticate('kuno', 'password')

The protobuf message for the robot state is given here:
https://dev.bostondynamics.com/protos/bosdyn/api/proto_reference#robot-state-proto

In [ ]:
state_client = robot.ensure_client('robot-state')
state_client.get_robot_state()

# Capture and view camera images

In [ ]:
from bosdyn.client.image import ImageClient
image_client = robot.ensure_client(ImageClient.default_service_name)
sources = image_client.list_image_sources()
[source.name for source in sources]
# This will show source names

image_respones = image_client.get_image_from_sources(["left_fisheye_image"])[0]
from PIL import Image
import io
image = Image.open(io.BytesIO(image_response.shot.image.data))
image.show()

# Configuring software E-Stop

Before spot's motors can be enabled, an independent "motor power authority" need to be setup. Settings are setup for allowing spot to automatically kill motor power for safety.

Here, it is a software E-Stop, not a hardware E-Stop.

In [ ]:
estop_client = robot.ensure_clien('estop')
estop_client.get_status()

An E-Stop "endpoint" can be created. This endstop provides a regular heartbeat to spot. If spot doesn't receive the heartbeat after a given timeout, it will power off.

It shouldn't be too small, otherwise network issues might cause a large enough delay for timeout.

In [ ]:
estop_endpoint = bosdyn.client.estop.EstopEndpoint(client=estop_client, name='my_estop', estop_timeout=9.0)
estop_endpoint.force_simple_setup()
# Then can get status again and see an endpoint is added
estop_client.get_status()

Creating the endstop above doesn't setup any process for regularly checking the E-Stop. 
To do this, an E-Stop client is created, passing the created endstop. 
This starts a background thread for reguarly checking the E-Stop.

In [ ]:
estop_keep_alive = bosdyn.client.estop.EstopKeepAlive(estop_endpoint)
estop_client.get_status()

# Taking ownership of Spot (leases)

Multiple clients can connect to spot, but only one can have control.  
To get control, a client needs to acquier a Lease.  
A valid lease must be provided by every mobility command to the robot.  
Like with authentication, the python API handles sending a lease with mobility commands.

In [ ]:
lease_client = robot.ensure_client('lease')
lease_client.list_leases()

In [ ]:
lease = lease_client.acquire()
lease_keep_alive = bosdyn.client.lease.LeaseKeepAlive(lease_client)
lease_client.list_leases()

A lease must be kept in scope for the entire duration of the program using the commands.  
The "body" lease takes ownership of the sub-resource "mobilibity".  
If spot also has an arm, this also takes owernship of the "full-arm", "arm" and "gripper" sub-resources.

# Powering on the robot

robot.power_on() is a helper function for a low-level power-on command, and returns once confirmation is received.

In [ ]:
robot.power_on(timeout_sec=20)
robot.is_powered_on()

# Timesync

The clocks on your machine and spot must be synced.  
This is such that commands are processed correctly, which come with a period of time for which they are valid, for safety.

The robot class maintains a timesync thread. The wait_for_sync call will start the timesync thread and isblocking.  
This thread is responsible for maintaining synchronisation between the two clocks. (Using the NTP protocol?)

In [ ]:
robot.time_sync.wait_for_sync()

# Commanding the robot

The "RobotCommandService" is the primary service for commanding mobility.  
Some available commands are:
- stand
- sit
- selfright
- safe_power_off
- velocity
- trajectory

In [ ]:
from bosdyn.client.robot_command import RobotCommandClient, blocking_stand

command_client = robot.ensure_client(RobotCommandClient.default_service_name)
blocking_stand(command_client, timeout_sec=10)

Can use the RobotCommandBuilder to execute more complex commands.  
eg: Specify a target pose of the body relative to the footprint.  
Orientation is specified in zxy instead of zyx, such that roll always occurs about a line on the xy plane.

In [ ]:
from bosdyn.geometry import EulerZXY
from bosdyn.client.robot_command import RobotCommandBuilder

# Stand up with a given orientation
cmd = RobotCommandBuilder.synchro_stand_command(
    footprint_R_body=EulerZXY(yaw=0.4, roll=0.0, pitch=0.0)
)
command_client.robot_command(cmd)
    
# Stand up with a given height
cmd = RobotCommandBuilder.synchro_stand_command(
    body_height=0.1
)
command_client.robot_command(cmd)

# Power off

Setting cut_immediately to False will cause spot to gradually come to a stop and sit down before powering off. Otherwise, it will cut motor power immediately and collapse.

In [ ]:
robt.power_off(cut_immediately=False)